In [ ]:
from PIL import Image
import numpy as np
from tifffile import imsave
import math
import matplotlib.pyplot as plt
import torch

FRAME_SIZE = 50
HALF_SIZE = int(FRAME_SIZE/2)
DECAY_RATE = 25
%matplotlib inline
Second_Layer_Kernel_Size = 151
CURVATURE = 10
    

def save_image_as_tiff(image,filename):
	image = np.einsum('ijk->kij', image)
	image = np.flip(image, axis=(1, 2))
	imsave(filename,image)

def Distance2Line(i, j, a, b, c):
    return abs(a * i + b * j + c) / math.sqrt(a**2 + b**2)

class LayerGeneration:
    def GenerateTopLayerWeights(angle, curvature):
        weight = np.zeros([FRAME_SIZE,FRAME_SIZE])
        a = math.cos(angle)
        b = math.sin(angle)
        c = -HALF_SIZE * (a + b)
        if curvature == 0:
            for i in range(FRAME_SIZE):
                for j in range(FRAME_SIZE):
                    dist = Distance2Line(i, j, a, b, c)
                    if (a * i + b * j + c < 0) and (dist < DECAY_RATE):
                        weight[i,j] = 1 - dist / DECAY_RATE
                    else:
                        weight[i,j] = 0 
        elif curvature > 0:
            r = (curvature * curvature + HALF_SIZE * HALF_SIZE) / (2 * curvature)
            x0 = (r * math.sqrt(a * a + b * b) + HALF_SIZE * b * b / a - HALF_SIZE * b - c) / (a + b * b / a)
            y0 = HALF_SIZE + (x0 - HALF_SIZE) * b / a
            for i in range(FRAME_SIZE):
                for j in range(FRAME_SIZE):
                    dist = math.sqrt((i - x0)**2 + (j - y0)**2)
                    if (dist > r - DECAY_RATE) and (dist < r):
                        weight[i,j] =  1 - (r - dist) / DECAY_RATE
                    else:
                        weight[i,j] = 0
        else:
            curvature = -curvature
            r = (curvature * curvature + 128 * 128) / (2 * curvature)
            x0 = (r * math.sqrt(a * a + b * b) + HALF_SIZE * b * b / a - HALF_SIZE * b - c) / (a + b * b / a)
            y0 = HALF_SIZE + (x0 - HALF_SIZE) * b / a
            for i in range(FRAME_SIZE):
                for j in range(FRAME_SIZE):
                    dist = math.sqrt((i - x0)**2 + (j - y0)**2)
                    if (dist < r + DECAY_RATE) and (dist > r):
                        weight[i,j] = 1 - (dist - r) / DECAY_RATE
                    else:
                        weight[i,j] = 0    
        return weight - np.mean(weight)
    
    
    def GenerateSecondLayerWeights(radius):
        L = np.arange(-(Second_Layer_Kernel_Size-1)/2, (Second_Layer_Kernel_Size-1)/2 + 1)
        L0 = np.arange(int(-Second_Layer_Kernel_Size/4), int(Second_Layer_Kernel_Size/4 + 1))
        Z, X, Y = np.meshgrid(L0, L, L)
        weight = X ** 2 + Y ** 2 + 4*(Z ** 2)
        weight = np.logical_and(weight <= radius ** 2, weight >= (radius-DECAY_RATE)**2).astype(float)
        return np.einsum('kij->ijk', weight*255)-np.mean(weight)
    
    
    def GenerateThirdLayerWeights(radius):
        L = np.arange(-radius, radius + 1)
        L0 = np.arange(int(-radius/2), int(radius/2 + 1))
        Z, X, Y = np.meshgrid(L0, L, L)
        weight = X ** 2 + Y ** 2 + 4*(Z ** 2)
        weight = np.logical_and(weight <= radius ** 2, weight >= (radius-DECAY_RATE)**2).astype(float)
        return np.einsum('kij->ijk', weight*255)-np.mean(weight)


def ReadDataFromTif(nframes, h, w):
    img_r = np.zeros((h, w, nframes))
    img_g = np.zeros((h, w, nframes))
    img_b = np.zeros((h, w, nframes))
    img_a = np.zeros((h, w, nframes))
    for i in range(nframes - 1):
        img.seek(i*4+1)
        img_r[:, :, i] = np.array(img)
        img.seek(i * 4 + 2)
        img_g[:, :, i] = np.array(img)
        img.seek(i * 4 + 3)
        img_b[:, :, i] = np.array(img)
        img.seek(i * 4 + 4)
        img_a[:, :, i] = np.array(img)
    img_rgba = np.max(np.stack([img_r, img_g, img_b, img_a]), axis=0)
    return img_rgba
        
def Read1FrameFromTif(nframes, h, w):
    img.seek(nframes * 4 + 1)
    img_r = np.array(img)
    img.seek(nframes * 4 + 2)
    img_g = np.array(img)
    img.seek(nframes * 4 + 3)
    img_b = np.array(img)
    img.seek(nframes * 4 + 4)
    img_a = np.array(img)
    img_rgba = np.max(np.stack([img_r, img_g, img_b, img_a]), axis=0)
    return img_rgba

def Layer1TorchConv(image,pattern,normalization):
    torch.cuda.empty_cache()
    #image = np.einsum('ijk->kij', image)
    pattern_tensor = torch.zeros(pattern.shape[0], 1, FRAME_SIZE, FRAME_SIZE)
    image_tensor = torch.zeros(1,1,h+Second_Layer_Kernel_Size-1,w+Second_Layer_Kernel_Size-1)
    image_tensor[0,0,:,:] = torch.from_numpy(image)
    pattern_tensor[:,0,:,:] = torch.from_numpy(pattern)
    layer1 = torch.nn.Conv2d(1, pattern.shape[0], (FRAME_SIZE, FRAME_SIZE)).to('cuda')
    layer1.load_state_dict({'weight': pattern_tensor}, strict=False)
    image_tensor=torch.nn.functional.pad(image_tensor, tuple((HALF_SIZE, HALF_SIZE, HALF_SIZE, HALF_SIZE)),'constant',value=0).to('cuda')
    out_layer1=layer1(image_tensor)
    if(normalization==True):
        pooling=torch.nn.AvgPool2d(FRAME_SIZE,stride=1)
        normalization_map = pooling(image_tensor)
        #print(normalization_map.size(),out_layer1.size())
        for kernal_frame in range(out_layer1.size()[1]):
            out_layer1[:,kernal_frame,:,:] = out_layer1[:,kernal_frame,:,:] - normalization_map * torch.sum(pattern_tensor[kernal_frame,:,:,:])
    out_layer1=torch.max(out_layer1,1).values.to('cpu')    
    return out_layer1.detach().numpy()

img = Image.open('C:/Users/brian/Documents/nTracer sample.tif')
h, w = np.shape(img)
nframes = int(img.n_frames/4)
image=Read1FrameFromTif(72,h,w)
image=image-np.min(image)-(np.max(image)-np.min(image))/2
fig=plt.figure()
plt.imshow(image)

In [ ]:
image=ReadDataFromTif(nframes,h,w)
image=image-np.min(image)
image=image/np.max(image)

pattern_distance_map=np.zeros((20*5,FRAME_SIZE,FRAME_SIZE))
for curvature in range(0,21,5):
    for angle in range(20):
        pattern_distance_map[angle + 4 * curvature, :, :] = LayerGeneration.GenerateTopLayerWeights(angle*0.314, curvature)
pd = ((int((Second_Layer_Kernel_Size-1)/2),int((Second_Layer_Kernel_Size-1)/2)),(int((Second_Layer_Kernel_Size-1)/2),int((Second_Layer_Kernel_Size-1)/2)),(int(Second_Layer_Kernel_Size/4),int(Second_Layer_Kernel_Size/4)))
image=np.pad(image,pd,mode='constant',constant_values = 0)
Layer1out = np.zeros((image.shape[2],1,image.shape[0]+1,image.shape[1]+1))
for frame in range(image.shape[2]):
    Layer1out_map = Layer1TorchConv(image[:,:,frame],pattern_distance_map,False)
    Layer1out[frame,:,:,:]=Layer1out_map
print('finished')

In [ ]:
Layer1outmask=Layer1out[:,0,:,:]
print(np.min(Layer1outmask),np.max(Layer1outmask))
Layer1outmask=Layer1outmask-np.min(Layer1outmask)-(np.max(Layer1outmask)-np.min(Layer1outmask))*3/10
Layer1outmask[Layer1outmask<0]=0

#Layer1outmask = np.flip(Layer1outmask, axis=(1, 2))
imsave('C:/Users/brian/Documents/Layer1ConvOut.tif',Layer1outmask)

In [ ]:
print(Layer1outmask.shape)
print(np.min(Layer1outmask),np.max(Layer1outmask))
plt.imshow(Layer1outmask[110,:,:])